# Function 6 - The Cake Recipe

### First Inspection

We are told that each feature quantifies the amount of five different ingredients that are used to make a cake. The output of the hidden black-box function $y = f(\mathbf{x})$ is a negative value that represents the taste of the cake. Smaller values allude to a worse taste while smaller values convey a better taste. Our objective is to maximise this function by bringing the score as close to zero as possible.

In [1]:
# Depedencies,
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# SKOPT imports,
from skopt import gp_minimize
from skopt.space import Real
from skopt.learning import GaussianProcessRegressor
from skopt.learning.gaussian_process.kernels import Matern, ConstantKernel, WhiteKernel
from skopt import Optimizer
from joblib import dump, load

# Loading known evaluations,
X, y = np.load("initial_inputs.npy"), np.load("initial_outputs.npy")

# Inspecting evalutation points,
X

array([[0.7281861 , 0.15469257, 0.73255167, 0.69399651, 0.05640131],
       [0.24238435, 0.84409997, 0.5778091 , 0.67902128, 0.50195289],
       [0.72952261, 0.7481062 , 0.67977464, 0.35655228, 0.67105368],
       [0.77062024, 0.11440374, 0.04677993, 0.64832428, 0.27354905],
       [0.6188123 , 0.33180214, 0.18728787, 0.75623847, 0.3288348 ],
       [0.78495809, 0.91068235, 0.7081201 , 0.95922543, 0.0049115 ],
       [0.14511079, 0.8966846 , 0.89632223, 0.72627154, 0.23627199],
       [0.94506907, 0.28845905, 0.97880576, 0.96165559, 0.59801594],
       [0.12572016, 0.86272469, 0.02854433, 0.24660527, 0.75120624],
       [0.75759436, 0.35583141, 0.0165229 , 0.4342072 , 0.11243304],
       [0.5367969 , 0.30878091, 0.41187929, 0.38822518, 0.5225283 ],
       [0.95773967, 0.23566857, 0.09914585, 0.15680593, 0.07131737],
       [0.6293079 , 0.80348368, 0.81140844, 0.04561319, 0.11062446],
       [0.02173531, 0.42808424, 0.83593944, 0.48948866, 0.51108173],
       [0.43934426, 0.69892383, 0.

Despite the sparsity of data points, the domain of the black box function seems to be $[0, 1]^5$.

In [3]:
# Inspecting black-box outputs,
y

array([-0.71426495, -1.20995524, -1.67219994, -1.53605771, -0.82923655,
       -1.24704893, -1.23378638, -1.69434344, -2.57116963, -1.30911635,
       -1.14478485, -1.91267714, -1.62283895, -1.35668211, -2.0184254 ,
       -1.70255784, -1.29424696, -0.93575656, -2.15576776, -1.74688209])

The output of the black-box function seem to always be negative and of the same magnitude.

### Optimiser Configuration

TBD.

### Optimiser Initialisation

In [5]:
"""INITALISING THE OPTIMISATION MODEL."""

# Inputting the given evaluations provided by the problem,
X_supplied = X.tolist()
y_supplied = y.tolist()

# Inital kappa value,
initial_kappa = 5

"""OPTIMISER SETTINGS."""

# We define the domain of the black-box function (or the range of the parameter values we want to consider),
space = [Real(0, 1, name="x1"),
         Real(0, 1, name="x2"),
         Real(0, 1, name="x3"),
         Real(0, 1, name="x4"),
         Real(0, 1, name="x5"),
         ]

# Creating the kernel for the GPR,
kernel = ConstantKernel(1.0) * Matern(
    length_scale=(0.1, 0.1, 0.1, 0.1, 0.1),
    length_scale_bounds=(1e-2, 1.0),
    nu = 5/2)

# GPR settings,
gpr = GaussianProcessRegressor(
    kernel=kernel,
    normalize_y=True,
    n_restarts_optimizer=10
)

# Creating optimisier,
opt = Optimizer(
    dimensions=space,
    base_estimator=gpr,
    acq_func="LCB",
    acq_func_kwargs={"kappa": initial_kappa},
    random_state=0
)

"""CREATING INTIAL OPTIMISER STATE."""

# Supplying given points to optimiser,
opt.tell(X_supplied, (-np.array(y_supplied)).tolist()) # <-- We flip the values since we are trying to maximise the black-box function.

# Asking for the next point to evaluate the black-box function,
point_query = opt.ask()

# Saving optimiser state (zero-th iteration),
dump(opt, "bayes_opt_state_iter0.joblib")

# Printing point query,
print(f"Point Query: {point_query}")

Point Query: [0.2851322613933646, 0.0, 0.5977831707171473, 1.0, 0.0]


### Next Query 

In [6]:
def kappa_decay(init_kappa, decay_param, iter_num):
    """Outputs a value of kappa given the current interaction number (current query number)."""
    return init_kappa*np.exp(-(iter_num/decay_param))

In [7]:
current_query = 1

# Input the new evaluation,
X_new = [[0.2851322613933646, 0.0, 0.5977831707171473, 1.0, 0.0]]
y_new = [-0.8219711662799899]

# Loading the previous state of the optimiser,
opt = load(f"bayes_opt_state_iter{current_query - 1}.joblib")

# Updating kappa,
new_kappa = kappa_decay(init_kappa=initial_kappa, decay_param=3.75, iter_num=current_query)
opt.acq_func_kwargs["kappa"] = new_kappa

# Supplying the new query to the optimiser,
opt.tell(X_new, (-np.array(y_new)).tolist()) # <-- We flip the values since we are trying to maximise the black-box function.

# Asking for the next point to evaluate the black-box function,
point_query = opt.ask()

# Saving optimiser state,
dump(opt, f"bayes_opt_state_iter{current_query}.joblib")

# Printing point query,
print(f"Point Query {current_query + 1}: {point_query}")

Point Query 2: [0.35840106518858067, 0.0, 1.0, 1.0, 1.0]


## Updates

Week 1: As expected, the optimiser selected point near the boundary of the feature space due to favouring exploration over exploitation with $\kappa = 5$. Not suprisingly, the next evaluation point is also on the boundary of the space due to kappa remaining high at $\kappa = 3.83$.